In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/12/31 20:02:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2024-12-31 04:12:49--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.126, 54.230.209.72, 54.230.209.200, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  58.5MB/s    in 5.2s    

2024-12-31 04:12:54 (56.3 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [4]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [49]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType

In [50]:
schema = StructType([
    StructField('hvfhs_license_num', StringType(), True), 
    StructField('dispatching_base_num', StringType(), True), 
    StructField('pickup_datetime', TimestampType(), True), 
    StructField('dropoff_datetime', TimestampType(), True), 
    StructField('PULocationID', IntegerType(), True), 
    StructField('DOLocationID', IntegerType(), True)])

In [51]:
df = spark.read \
    .options(header=True, schema = schema)\
    .parquet('fhvhv_tripdata_2021-01.parquet')\
    .select('hvfhs_license_num', 'dispatching_base_num'
            , 'pickup_datetime', 'dropoff_datetime'
           , 'PULocationID', 'DOLocationID')

In [52]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True)])

In [20]:
# !head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet

In [53]:
df = df.repartition(24) #Manually partition the file to 24 blocks

In [54]:
df.write.parquet('fhvhv/2021/01/')

In [5]:
df = spark.read.format('parquet').load('fhvhv/2021/01/')

In [8]:
# Schema Commands
df.schema
df.printSchema()

+-----------------+--------------------+-------------------+-------------------+------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-----------------+--------------------+-------------------+-------------------+------------+------------+
|           HV0003|              B02870|2021-01-20 14:18:56|2021-01-20 14:41:03|          47|          74|
|           HV0003|              B02872|2021-01-28 08:33:55|2021-01-28 08:45:46|         229|         170|
|           HV0005|              B02510|2021-01-09 01:57:17|2021-01-09 02:27:17|           3|          76|
|           HV0005|              B02510|2021-01-29 09:21:56|2021-01-29 09:40:04|          89|          35|
|           HV0003|              B02875|2021-01-01 01:29:30|2021-01-01 01:48:48|          61|          62|
|           HV0003|              B02875|2021-01-15 03:20:58|2021-01-15 03:39:30|          76|         225|
|           HV0003|              B028

In [ ]:
# Output Commands
.show()
.take(num_records)
.head()
.write

In [ ]:
# Spark Functions
from pyspark.sql import functions as F
F.to_date()
F.udf()

In [ ]:
#UDF is abbreviation of User Defined Function
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

crazy_stuff_udf = F.udf(crazy_stuff, returnType = StringType())

df\
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))